# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import pickle
import nltk

nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

import warnings

warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///Messages.db')
df = pd.read_sql("SELECT * FROM Messages", engine)

X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    """Normalize, tokenize and stem text string
    
    Args:
    text: string. String containing message for processing
       
    Returns:
    stemmed: list of strings. List containing normalized and stemmed word tokens
    """
    # Convert text to lowercase and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize words
    tokens = word_tokenize(text)
    
    # Stem word tokens and remove stop words
    stemmer = PorterStemmer()
    stop_words = stopwords.words("english")
    
    stemmed = [stemmer.stem(word) for word in tokens if word not in stop_words]
    
    return stemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
from sklearn.multioutput import MultiOutputClassifier

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=71)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
def get_eval_metrics(actual, predicted, col_names):
    """Calculate evaluation metrics for ML model
    
    Args:
    actual: array. Array containing actual labels.
    predicted: array. Array containing predicted labels.
    col_names: list of strings. List containing names for each of the predicted fields.
       
    Returns:
    metrics_df: dataframe. Dataframe containing the accuracy, precision, recall 
    and f1 score for a given set of actual and predicted labels.
    """
    metrics = []
    
    # Calculate evaluation metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(actual[:, i], predicted[:, i])
        precision = precision_score(actual[:, i], predicted[:, i],average='micro')
        recall = recall_score(actual[:, i], predicted[:, i],average='micro')
        f1 = f1_score(actual[:, i], predicted[:, i],average='micro')
        
        metrics.append([accuracy, precision, recall, f1])
    
    # Create dataframe containing metrics
    metrics = np.array(metrics)
    metrics_df = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return metrics_df

In [7]:
Y_train_pred = pipeline.predict(X_train)
col_names = list(Y.columns.values)

print(get_eval_metrics(np.array(Y_train), Y_train_pred, col_names))


                        Accuracy  Precision    Recall        F1
related                 0.990654   0.990654  0.990654  0.990654
request                 0.986935   0.986935  0.986935  0.986935
offer                   0.998665   0.998665  0.998665  0.998665
aid_related             0.984980   0.984980  0.984980  0.984980
medical_help            0.988270   0.988270  0.988270  0.988270
medical_products        0.991465   0.991465  0.991465  0.991465
search_and_rescue       0.993849   0.993849  0.993849  0.993849
security                0.994707   0.994707  0.994707  0.994707
military                0.996138   0.996138  0.996138  0.996138
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.994469   0.994469  0.994469  0.994469
food                    0.994373   0.994373  0.994373  0.994373
shelter                 0.991799   0.991799  0.991799  0.991799
clothing                0.997711   0.997711  0.997711  0.997711
money                   0.996138   0.996

In [8]:
# Calculate evaluation metrics for test set
Y_test_pred = pipeline.predict(X_test)

eval_metrics0 = get_eval_metrics(np.array(Y_test), Y_test_pred, col_names)
print(eval_metrics0)

                        Accuracy  Precision    Recall        F1
related                 0.807399   0.807399  0.807399  0.807399
request                 0.888635   0.888635  0.888635  0.888635
offer                   0.995423   0.995423  0.995423  0.995423
aid_related             0.751144   0.751144  0.751144  0.751144
medical_help            0.928871   0.928871  0.928871  0.928871
medical_products        0.953852   0.953852  0.953852  0.953852
search_and_rescue       0.970633   0.970633  0.970633  0.970633
security                0.981503   0.981503  0.981503  0.981503
military                0.966629   0.966629  0.966629  0.966629
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.950610   0.950610  0.950610  0.950610
food                    0.938978   0.938978  0.938978  0.938978
shelter                 0.934401   0.934401  0.934401  0.934401
clothing                0.985889   0.985889  0.985889  0.985889
money                   0.978261   0.978

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
def performance_metric(y_true, y_pred):
    """Calculate median F1 score for all of the output classifiers
    
    Args:
    y_true: array. Array containing actual labels.
    y_pred: array. Array containing predicted labels.
        
    Returns:
    score: float. Median F1 score for all of the output classifiers
    """
    f1_list = []
    for i in range(np.shape(y_pred)[1]):
        f1 = f1_score(np.array(y_true)[:, i], y_pred[:, i], average='micro')
        f1_list.append(f1)
        
    score = np.median(f1_list)
    return score
parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[2, 5, 10]}

scorer = make_scorer(performance_metric)
cv = GridSearchCV(pipeline, param_grid = parameters, scoring = scorer, verbose = 10)

# Find best parameters
np.random.seed(81)
tuned_model = cv.fit(X_train, Y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.9578028894292662, total= 1.1min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.4min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.9591617794306966, total= 1.1min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.8min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1, score=0.9553648068669527, total= 1.1min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed: 10.6min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=1, score=0.9557939914163089, total= 1.1min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 12.0min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.9596624231154341, total=  53.2s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.960449148905736, total=  53.3s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.9587267525035765, total=  52.9s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=1, score=0.9598054641682163, total= 2.0min
[CV] clf__estimator__min_samples_split=2, clf__e

[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed: 117.0min finished


In [10]:

# Get results of grid search
tuned_model.cv_results_

{'mean_fit_time': array([  53.9573342 ,   44.3698562 ,   55.05597059,   43.76403928,
         110.23458242,   86.67768375,  112.85497705,   85.04113658,
          46.11938691,   40.73215914,   45.85723472,   39.29702584,
          89.73457734,   77.22345265,   89.91035461,   74.42761525,
          43.29196143,   39.31711245,   42.67355021,   37.89729643,
          82.88600159,   73.65914329,   81.25589609,   70.59951758]),
 'std_fit_time': array([ 0.5798039 ,  0.42277498,  1.0541529 ,  0.28809062,  1.00190523,
         0.63061587,  1.28348658,  0.62502073,  0.17015837,  0.21670168,
         0.2470712 ,  0.28324009,  0.30669932,  0.39586694,  0.28401311,
         0.21909037,  0.18096546,  0.33766019,  0.45599212,  0.17314726,
         0.32330257,  0.38148161,  0.30885475,  0.78499402]),
 'mean_score_time': array([  9.8821497 ,   9.4619561 ,   9.81571182,   9.39157216,
         12.49557217,  11.3044459 ,  12.37783909,  11.45710595,
          9.85655427,   9.44431909,   9.71644648,   9.28

In [11]:

# Best mean test score
np.max(tuned_model.cv_results_['mean_test_score'])

0.96244993324432582

In [12]:
# Parameters for best mean test score
tuned_model.best_params_

{'clf__estimator__min_samples_split': 10,
 'clf__estimator__n_estimators': 25,
 'tfidf__use_idf': True,
 'vect__min_df': 5}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:

# Calculate evaluation metrics for test set
tuned_pred_test = tuned_model.predict(X_test)

eval_metrics1 = get_eval_metrics(np.array(Y_test), tuned_pred_test, col_names)

print(eval_metrics1)

                        Accuracy  Precision    Recall        F1
related                 0.818841   0.818841  0.818841  0.818841
request                 0.892830   0.892830  0.892830  0.892830
offer                   0.995423   0.995423  0.995423  0.995423
aid_related             0.780320   0.780320  0.780320  0.780320
medical_help            0.930015   0.930015  0.930015  0.930015
medical_products        0.956903   0.956903  0.956903  0.956903
search_and_rescue       0.970633   0.970633  0.970633  0.970633
security                0.981503   0.981503  0.981503  0.981503
military                0.966629   0.966629  0.966629  0.966629
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.963959   0.963959  0.963959  0.963959
food                    0.952899   0.952899  0.952899  0.952899
shelter                 0.946606   0.946606  0.946606  0.946606
clothing                0.985698   0.985698  0.985698  0.985698
money                   0.977689   0.977

In [14]:
# Get summary stats for first model
eval_metrics0.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.945297,0.945297,0.945297,0.945297
std,0.056666,0.056666,0.056666,0.056666
min,0.751144,0.751144,0.751144,0.751144
25%,0.937834,0.937834,0.937834,0.937834
50%,0.960336,0.960336,0.960336,0.960336
75%,0.982599,0.982599,0.982599,0.982599
max,1.000000,1.000000,1.000000,1.000000


In [15]:

# Get summary stats for tuned model
eval_metrics1.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.950165,0.950165,0.950165,0.950165
std,0.051167,0.051167,0.051167,0.051167
min,0.780320,0.780320,0.780320,0.780320
25%,0.945748,0.945748,0.945748,0.945748
50%,0.964245,0.964245,0.964245,0.964245
75%,0.983696,0.983696,0.983696,0.983696
max,1.000000,1.000000,1.000000,1.000000


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [16]:
from sklearn.naive_bayes import BernoulliNB

pipeline2 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(BernoulliNB()))
    ])

parameters2 = {
                'clf__estimator__fit_prior': [True, False],
                'clf__estimator__alpha': [0, 1, 2],
              }

cv2 = GridSearchCV(pipeline2, param_grid=parameters2)
new_model2 = cv2.fit(X_train, Y_train)
new_model2.cv_results_

{'mean_fit_time': array([ 16.34094572,  16.20881581,  16.39457067,  16.26617169,
         16.38575109,  16.60354106]),
 'std_fit_time': array([ 0.12218311,  0.08752283,  0.15167826,  0.11741955,  0.0499043 ,
         0.26169727]),
 'mean_score_time': array([ 8.23167491,  8.14754725,  8.19075632,  8.05127184,  8.08176438,
         8.14498075]),
 'std_score_time': array([ 0.1149151 ,  0.12167545,  0.24817801,  0.21684216,  0.24388543,
         0.06132707]),
 'param_clf__estimator__alpha': masked_array(data = [0 0 1 1 2 2],
              mask = [False False False False False False],
        fill_value = ?),
 'param_clf__estimator__fit_prior': masked_array(data = [True False True False True False],
              mask = [False False False False False False],
        fill_value = ?),
 'params': [{'clf__estimator__alpha': 0, 'clf__estimator__fit_prior': True},
  {'clf__estimator__alpha': 0, 'clf__estimator__fit_prior': False},
  {'clf__estimator__alpha': 1, 'clf__estimator__fit_prior': True},

In [17]:
new_model2.best_params_

{'clf__estimator__alpha': 1, 'clf__estimator__fit_prior': True}

In [18]:
new_Y_pred2 = new_model2.predict(X_test)

new_metrics2 = []
for i in range(len(Y_test.columns)):
    accuracy = accuracy_score(Y_test.iloc[:, i], new_Y_pred2[:, i])
    precision = precision_score(Y_test.iloc[:, i], new_Y_pred2[:, i], average='micro')
    recall = recall_score(Y_test.iloc[:, i], new_Y_pred2[:, i], average='micro')
    f1 = f1_score(Y_test.iloc[:, i], new_Y_pred2[:, i], average='micro')
    new_metrics2.append([accuracy, precision, recall, f1])

new_metrics_df2 = pd.DataFrame(data=np.array(new_metrics2), index=list(Y_test.columns.values), columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
print(new_metrics_df2)

                        Accuracy  Precision    Recall        F1
related                 0.816171   0.816171  0.816171  0.816171
request                 0.885011   0.885011  0.885011  0.885011
offer                   0.992944   0.992944  0.992944  0.992944
aid_related             0.754958   0.754958  0.754958  0.754958
medical_help            0.921243   0.921243  0.921243  0.921243
medical_products        0.949085   0.949085  0.949085  0.949085
search_and_rescue       0.968154   0.968154  0.968154  0.968154
security                0.979786   0.979786  0.979786  0.979786
military                0.963387   0.963387  0.963387  0.963387
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.935355   0.935355  0.935355  0.935355
food                    0.907132   0.907132  0.907132  0.907132
shelter                 0.919718   0.919718  0.919718  0.919718
clothing                0.983028   0.983028  0.983028  0.983028
money                   0.974447   0.974

In [19]:
new_metrics_df2.describe()

,Accuracy,Precision,Recall,F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.937648,0.937648,0.937648,0.937648
std,0.058552,0.058552,0.058552,0.058552
min,0.754958,0.754958,0.754958,0.754958
25%,0.918669,0.918669,0.918669,0.918669
50%,0.953947,0.953947,0.953947,0.953947
75%,0.980597,0.980597,0.980597,0.980597
max,1.000000,1.000000,1.000000,1.000000


### 9. Export your model as a pickle file

In [21]:
pickle.dump(tuned_model, open('disaster_pipeline.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.